In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Whispering-GPT/lex-fridman-podcast")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/346 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Convert the "train" split to a pandas DataFrame
df = pd.DataFrame(dataset["train"])

print(df.head())

            id      channel                channel_id  \
0  TRdL6ZzWBS0  Lex Fridman  UCSHZKyawb77ixDdsGog4iWA   
1  TPXTmVdlyoc  Lex Fridman  UCSHZKyawb77ixDdsGog4iWA   
2  -t1_ffaFXao  Lex Fridman  UCSHZKyawb77ixDdsGog4iWA   
3  BCdV6BMMpOo  Lex Fridman  UCSHZKyawb77ixDdsGog4iWA   
4  Kedt2or9xlo  Lex Fridman  UCSHZKyawb77ixDdsGog4iWA   

                                               title              categories  \
0  Jed Buchwald: Isaac Newton and the Philosophy ...  [Science & Technology]   
1  Sergey Nazarov: Chainlink, Smart Contracts, an...  [Science & Technology]   
2  Stephen Wolfram: Fundamental Theory of Physics...  [Science & Technology]   
3  Philip Goff: Consciousness, Panpsychism, and t...  [Science & Technology]   
4  Oriol Vinyals: DeepMind AlphaStar, StarCraft, ...  [Science & Technology]   

                                                tags  \
0  [agi, ai, ai podcast, artificial intelligence,...   
1  [agi, ai, ai podcast, artificial intelligence,...   
2  [step

In [ ]:
df['full_text'] = df['segments'].apply(lambda segments: ' '.join(segment['text'] for segment in segments))
df['segment_details'] = df['segments'].apply(lambda segments: [(segment['start'], segment['end'], segment['text']) for segment in segments])
df_new = df[['full_text', 'segment_details']].copy()
print(df_new.head())


                                           full_text  \
0   The following is a conversation with Jed Buck...   
1   The following is a conversation with Sergey N...   
2   The following is a conversation with Stephen ...   
3   I believe our official scientific worldview i...   
4   The following is a conversation with Ariol Vi...   

                                     segment_details  
0  [(0.0, 7.68,  The following is a conversation ...  
1  [(0.0, 6.5,  The following is a conversation w...  
2  [(0.0, 4.48,  The following is a conversation ...  
3  [(0.0, 5.2,  I believe our official scientific...  
4  [(0.0, 3.2800000000000002,  The following is a...  


In [ ]:
df_new['full_text'][0]

" The following is a conversation with Jed Buckwald, a professor of history and a philosopher of science at Caltech.  Interested especially in the development of scientific concepts and the instruments used to create and explore new effects and ideas in science.  To support this podcast, please check out our sponsors in the description.  This is the Lex Friedman podcast and here is my conversation with Jed Buckwald.  Does science progress via paradigm shifts and revolutions as philosopher Thomas Kuhn said, or does it progress gradually? What do you think?  Well, I got into this field because I was Tom Kuhn's research assistant 50 years ago, 52 years ago.  He pulled me into it out of physics instead. So I know his work pretty well.  And in the years when I was at MIT running an institute, he was then in the philosophy department, used to come over all the time to the talks we held and so on.  So what would I say about that? He, of course, developed his ideas a lot over the years.  The t

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import nltk
import string
import numpy as np
import math
from collections import defaultdict
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
import sys
import os
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words_lists = ['a', 'the', 'an', 'and', 'or', 'but', 'about', 'above', 'after', 'along', 'amid', 'among',
                    'as', 'at', 'by', 'for', 'from', 'in', 'into', 'like', 'minus', 'near', 'of', 'off', 'on',
                    'onto', 'out', 'over', 'past', 'per', 'plus', 'since', 'till', 'to', 'under', 'until', 'up',
                    'via', 'vs', 'with', 'that', 'can', 'cannot', 'could', 'may', 'might', 'must',
                    'need', 'ought', 'shall', 'should', 'will', 'would', 'have', 'had', 'has', 'having', 'be',
                    'is', 'am', 'are', 'was', 'were', 'being', 'been', 'get', 'gets', 'got', 'gotten',
                    'getting', 'seem', 'seeming', 'seems', 'seemed',
                    'enough', 'both', 'all', 'your' 'those', 'this', 'these',
                    'their', 'the', 'that', 'some', 'our', 'no', 'neither', 'my',
                    'its', 'his' 'her', 'every', 'either', 'each', 'any', 'another',
                    'an', 'a', 'just', 'mere', 'such', 'merely' 'right', 'no', 'not',
                    'only', 'sheer', 'even', 'especially', 'namely', 'as', 'more',
                    'most', 'less' 'least', 'so', 'enough', 'too', 'pretty', 'quite',
                    'rather', 'somewhat', 'sufficiently' 'same', 'different', 'such',
                    'when', 'why', 'where', 'how', 'what', 'who', 'whom', 'which',
                    'whether', 'why', 'whose', 'if', 'anybody', 'anyone', 'anyplace',
                    'anything', 'anytime' 'anywhere', 'everybody', 'everyday',
                    'everyone', 'everyplace', 'everything' 'everywhere', 'whatever',
                    'whenever', 'whereever', 'whichever', 'whoever', 'whomever' 'he',
                    'him', 'his', 'her', 'she', 'it', 'they', 'them', 'its', 'their', 'theirs',
                    'you', 'your', 'yours', 'me', 'my', 'mine', 'I', 'we', 'us', 'much', 'and/or','...', '—', '’', '“', '”', '‘', '…'
                    ]

all_stopwords = set(stopwords.words('english')).union(stop_words_lists)
stemmer = nltk.stem.snowball.EnglishStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
stemmer = PorterStemmer()

def token_preprocess(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Filter out punctuation and stop words, then stem
    processed_tokens = [
        stemmer.stem(word.lower()) for word in tokens
        if word.isalpha() and word.lower() not in all_stopwords
    ]
    return processed_tokens

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=token_preprocess)
tfidf_matrix = vectorizer.fit_transform(df['full_text'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
sim_df = pd.DataFrame(cosine_sim)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
print("Cosine Similarity Matrix:\n", sim_df)

Cosine Similarity Matrix:
           0         1         2         3         4         5         6    \
0    1.000000  0.299567  0.463112  0.379948  0.287810  0.447092  0.416021   
1    0.299567  1.000000  0.348772  0.358657  0.294873  0.424918  0.438545   
2    0.463112  0.348772  1.000000  0.468978  0.356818  0.508855  0.486266   
3    0.379948  0.358657  0.468978  1.000000  0.329741  0.430072  0.473091   
4    0.287810  0.294873  0.356818  0.329741  1.000000  0.366895  0.366751   
..        ...       ...       ...       ...       ...       ...       ...   
341  0.351427  0.321054  0.378316  0.379419  0.306423  0.431334  0.469512   
342  0.352516  0.329025  0.404872  0.411920  0.321401  0.478844  0.487166   
343  0.421590  0.411069  0.546904  0.469962  0.376167  0.602758  0.641721   
344  0.288726  0.292314  0.339862  0.300180  0.451059  0.363350  0.349597   
345  0.396663  0.367443  0.560521  0.473244  0.381542  0.536410  0.562110   

          7         8         9    ...       336

In [ ]:
for i in range(len(df)):
    # Sort the similarities for the ith episode, excluding self-similarity at index i
    sorted_indices = sim_df.iloc[i].drop(i).sort_values(ascending=False).index
    most_similar = sorted_indices[0]
    print(f"Episode {i} is most similar to Episode {most_similar} with a similarity score of {sim_df.iloc[i, most_similar]:.2f}.")

Episode 0 is most similar to Episode 222 with a similarity score of 0.52.
Episode 1 is most similar to Episode 221 with a similarity score of 0.55.
Episode 2 is most similar to Episode 192 with a similarity score of 0.84.
Episode 3 is most similar to Episode 273 with a similarity score of 0.73.
Episode 4 is most similar to Episode 191 with a similarity score of 0.53.
Episode 5 is most similar to Episode 250 with a similarity score of 0.67.
Episode 6 is most similar to Episode 202 with a similarity score of 0.77.
Episode 7 is most similar to Episode 201 with a similarity score of 0.66.
Episode 8 is most similar to Episode 308 with a similarity score of 0.71.
Episode 9 is most similar to Episode 214 with a similarity score of 0.77.
Episode 10 is most similar to Episode 211 with a similarity score of 0.70.
Episode 11 is most similar to Episode 322 with a similarity score of 0.71.
Episode 12 is most similar to Episode 156 with a similarity score of 0.70.
Episode 13 is most similar to Episo

In [ ]:
threshold = 0.8
for i in range(len(df)):
    similar_episodes = sim_df.iloc[i].where(lambda x: (x >= threshold) & (x < 1.0)).dropna()
    for j, score in similar_episodes.items():
        if i != j:
            print(f"Episode {i} and Episode {j} score {score:.2f}")


Episode 2 and Episode 57 score 0.84
Episode 2 and Episode 192 score 0.84
Episode 41 and Episode 283 score 0.82
Episode 57 and Episode 2 score 0.84
Episode 57 and Episode 192 score 0.83
Episode 68 and Episode 143 score 0.81
Episode 68 and Episode 254 score 0.84
Episode 131 and Episode 210 score 0.81
Episode 131 and Episode 330 score 0.84
Episode 143 and Episode 68 score 0.81
Episode 154 and Episode 190 score 0.81
Episode 190 and Episode 154 score 0.81
Episode 192 and Episode 2 score 0.84
Episode 192 and Episode 57 score 0.83
Episode 210 and Episode 131 score 0.81
Episode 210 and Episode 330 score 0.81
Episode 254 and Episode 68 score 0.84
Episode 283 and Episode 41 score 0.82
Episode 330 and Episode 131 score 0.84
Episode 330 and Episode 210 score 0.81


In [ ]:
threshold = 0.8
similar_pairs = set()


for i in range(len(sim_df)):
    for j in range(i + 1, len(sim_df)):
        if sim_df.iloc[i, j] >= threshold:
            similar_pairs.add((i, j))

for i, j in similar_pairs:
    tags_i = df.at[i, 'tags']
    tags_j = df.at[j, 'tags']

    print(f"Episode {i} and Episode {j} are similar.")
    print(f"Episode {i} Tags: {tags_i}")
    print(f"Episode {j} Tags: {tags_j}\n")


Episode 154 and Episode 190 are similar.
Episode 154 Tags: ['agi', 'ai', 'ai podcast', 'artificial intelligence', 'artificial intelligence podcast', 'automation', 'ethics', 'jeffrey epstein', 'kate darling', 'lex ai', 'lex fridman', 'lex jre', 'lex mit', 'lex podcast', 'mit', 'mit ai', 'privacy', 'robots']
Episode 190 Tags: ['kate darling', 'mit', 'media lab', 'hri', 'human-robot interaction', 'artificial intelligence', 'agi', 'ai', 'ai podcast', 'artificial intelligence podcast', 'lex fridman', 'lex podcast', 'lex mit', 'lex ai', 'lex jre', 'mit ai']

Episode 41 and Episode 283 are similar.
Episode 41 Tags: ['Judo', 'Karelin', 'Kayla Harrison', 'Putin', 'Ronda Rousey', 'Travis Stevens', 'agi', 'ai', 'ai podcast', 'artificial intelligence', 'artificial intelligence podcast', 'championship', 'coach', 'jimmy pedro', 'khabib', 'lex ai', 'lex fridman', 'lex jre', 'lex mit', 'lex podcast', 'medals', 'mit ai', 'national team', 'olympic games', 'olympics', 'randori', 'seoi nage', 'shohei ono'

In [ ]:
for i, j in similar_pairs:
    tags_i = set(df.at[i, 'tags'])
    tags_j = set(df.at[j, 'tags'])

    intersection = tags_i & tags_j
    union = tags_i | tags_j
    jaccard_similarity = len(intersection) / len(union) if union else 0

    print(f"Episode {i} and Episode {j} share {len(intersection)} tags out of {len(union)} total unique tags. Jaccard Similarity: {jaccard_similarity:.2f}")

Episode 154 and Episode 190 share 13 tags out of 21 total unique tags. Jaccard Similarity: 0.62
Episode 41 and Episode 283 share 15 tags out of 57 total unique tags. Jaccard Similarity: 0.26
Episode 131 and Episode 210 share 12 tags out of 28 total unique tags. Jaccard Similarity: 0.43
Episode 68 and Episode 254 share 12 tags out of 12 total unique tags. Jaccard Similarity: 1.00
Episode 2 and Episode 192 share 12 tags out of 17 total unique tags. Jaccard Similarity: 0.71
Episode 2 and Episode 57 share 12 tags out of 25 total unique tags. Jaccard Similarity: 0.48
Episode 57 and Episode 192 share 13 tags out of 29 total unique tags. Jaccard Similarity: 0.45
Episode 210 and Episode 330 share 12 tags out of 24 total unique tags. Jaccard Similarity: 0.50
Episode 68 and Episode 143 share 11 tags out of 13 total unique tags. Jaccard Similarity: 0.85
Episode 131 and Episode 330 share 14 tags out of 26 total unique tags. Jaccard Similarity: 0.54


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def preprocess_text2(text):
    text = re.sub(r'\W', ' ', str(text))
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
    return ' '.join(tokens)

df_new['processed_text'] = df_new['full_text'].apply(preprocess_text2)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_new['processed_text'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def find_relevant_answer(query):
    query_vec = vectorizer.transform([preprocess_text2(query)])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    max_index = similarities.argmax()
    return df.iloc[max_index]['full_text']

user_query = "What is Bitcoin"
response = find_relevant_answer(user_query)
print("Chatbot:", response[:500])


Chatbot:  The following is a conversation with Nick Carter, who is a partner at Castle Island Ventures,  co-founder of CoinMetrics.io, and previously a crypto asset research analyst at Fidelity Investments.  He's a prominent writer, speaker, and podcaster on topics around decentralized finance and especially  Bitcoin. Quick mention of our sponsors. The Information, Athletic Greens, Four Sigmatic,  and Blinkist. Check them out in the description to support this podcast.  This conversation with Nick Carter


In [ ]:
def extract_relevant_part(full_text, query):
    sentences = nltk.sent_tokenize(full_text)
    best_sentence = ''
    max_sim = 0
    query_vec = vectorizer.transform([preprocess_text2(query)])
    for sentence in sentences:
        sentence_vec = vectorizer.transform([preprocess_text2(sentence)])
        sim = cosine_similarity(query_vec, sentence_vec)[0][0]
        if sim > max_sim:
            max_sim = sim
            best_sentence = sentence
    return best_sentence

response_sentence = extract_relevant_part(response, user_query)
print("Chatbot:", response_sentence)


Chatbot: That is Bitcoin.
